In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.webdriver.chrome.options import Options

# Set up WebDriver (Ensure you have installed ChromeDriver)
chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")  # Blocks popups
chrome_options.add_argument("--disable-notifications")  # Blocks notification popups
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])

In [2]:
def get_rns_urls(trust):

    # Launch WebDriver with options
    driver = webdriver.Chrome(options=chrome_options)

    url = f"https://www.lse.co.uk/rns/{trust}/?filter=holdings-in-company"

    driver.get(url)

    # Wait for cookie popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
        )
        cookie_button.click()
        # print("Accepted cookies.")
    except:
        # print("No cookie popup found or already accepted.")
        pass

    # Wait for table to load
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        html_content = driver.page_source  # Get fully rendered HTML
        soup = BeautifulSoup(html_content, "html.parser")
        # Select tbody
        tbody = soup.find("tbody")

        # Extract rows from tbody
        rows = tbody.find_all("tr") if tbody else []

        rns_urls = []

        # Loop through rows and extract cell data
        for row in rows:
            cells = row.find_all("td")
            cell_texts = [cell.text.strip() for cell in cells]
            for td in cells:
                a_tag = td.find("a", href=True)  # Find <a> inside <td>
                if a_tag:
                    rns_url = urljoin("https://www.lse.co.uk", a_tag["href"])  # Construct full URL
                    rns_urls.append(rns_url)
                    # print(rns_url)
            # print(cell_texts, rns_url)

            # Open the link in Selenium
            # driver.get(rns_url)

            # try:
            #     cookie_button = WebDriverWait(driver, 5).until(
            #         EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
            #     )
            #     cookie_button.click()
            #     # print("Accepted cookies.")
            # except:
            #     # print("No cookie popup found or already accepted.")
            #     pass

            # # Wait for investor popup and accept it
            # try:
            #     cookie_button = WebDriverWait(driver, 5).until(
            #         EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Private Investor')]"))
            #     )
            #     cookie_button.click()
            #     # print("Accepted private investor.")
            # except:
            #     # print("No private investor popup found or already accepted.")
            #     pass

            # try: 
            #     # Wait for page load
            #     WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            # except:
            #     print("RNS Page took too long to load.")
            # # # Check if "Saba" is in the page source
            # # page_source = driver.page_source
            # # if "Saba" in page_source:
            # #     print(f"Found 'Saba' in: {rns_url}")
            # #     break
        
        return rns_urls

    except:
        print("Table not found or page took too long to load.")

    driver.quit()

def check_rns_content(rns_url):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(rns_url)

    # Wait for cookie popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
        )
        cookie_button.click()
        # print("Accepted cookies.")
    except:
        # print("No cookie popup found or already accepted.")
        pass

    # Wait for investor popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Private Investor')]"))
        )
        cookie_button.click()
        # print("Accepted private investor.")
    except:
        # print("No private investor popup found or already accepted.")
        pass

    try: 
        # Wait for page load
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except:
        print("RNS Page took too long to load.")

    # Check if "Saba" is in the page source
    page_source = driver.page_source
    if "Saba" in page_source:
        print(f"Found 'Saba' in: {rns_url}")
        driver.quit()

        return True
    driver.quit()

    return False


# rns_urls = get_rns_urls("EWI")

# for rns_url in rns_urls:
#     if check_rns_content(rns_url):
#         break


In [3]:
done = ["AAIF","AAS","ADIG","AEI","AGT","AGVI","AIE","AJOT","ALW","AMAT","ANII","ARR","ASL","ATR","ATT","ATY","AUSC","AWEM","BAF","BASC","BBH","BEMO","BERI","BGCG","BGEU","BGFD","BGS","BGUK","BIOG","BNKR","BPCP","BPCR","BRAI","BRFI","BRGE","BRIG","BRLA","BRSC","BRWM","BUT","CCJI","CGI","CGT","CHI","CHIB","CLC","CLDN","CMPG","CMPI","CRT","CTUK","CTY","CYN","DGN","DIG","DIVI","EAT","EDIN","ENRG","EOT","ESCT","FA17","FAS","FCIT","FCSS","FEML","FEV","FGEN","FGT","FJV","FSV","FVEN","GCL","GPM","GSCT","HAN","HANA","HET","HFEL","HGEN","HHI","HHV","HINT","HOT","HRI","HSL","HVPE","HYG","IAT","IEM","IGC","IGET","INV","IPU","JAGI","JAM","JCGI","JCH","JEDT","JEGI","JEMA","JEMI","JFJ","JGC","JGGI","JII",]
trusts = ["JMG","JUGI","JUSC","KPC","LTI","LWDB","LWI","MCT","MHN","MIGO","MINI","MIX","MMIT","MNKS","MNL","MNP","MRC","MRCH","MTE","MTU","MUT","MWY","MYI","NAIT","NAS","NAVF","NSI","OIG","OIT","ONWD","OOA","OSEC","OVCT","PAC","PCFT","PCGH","PCT","PGV","PHI","PNL","PSH","RCOI","RICA","RIII","RKW","RMMC","SAIN","SCF","SCP","SDP","SDV","SEC","SHRS","SIHL","SJG","SMT","SOI","SSON","SST","STS","SVM","TEM","TENT","THRG","TMPL","TRY","UEM","USA","VEIL","VNH","WINV","WWH"]
results = {}
for trust in trusts:
    rns_urls = get_rns_urls(trust)
    results[trust] = False
    for rns_url in rns_urls:
        if check_rns_content(rns_url):
            results[trust] = True
            break

Found 'Saba' in: https://www.lse.co.uk/rns/JUGI/holdings-in-company-ox312z21hv1ahcb.html
Found 'Saba' in: https://www.lse.co.uk/rns/KPC/holdings-in-company-xj27qkshm4sjtia.html
Found 'Saba' in: https://www.lse.co.uk/rns/LWI/holdings-in-company-ougropm7z3auh58.html
Found 'Saba' in: https://www.lse.co.uk/rns/MCT/middlefield-canadian-income-pcc-holdings-in-company-hc67h4eg38h7rep.html
Found 'Saba' in: https://www.lse.co.uk/rns/MTE/holdings-in-company-4twvb2ysfrdr2wr.html
Found 'Saba' in: https://www.lse.co.uk/rns/MTU/holdings-in-company-pzk1n1q51i99z77.html
Found 'Saba' in: https://www.lse.co.uk/rns/NAIT/holdings-in-company-4j5jrxr6krbdjqr.html
Found 'Saba' in: https://www.lse.co.uk/rns/RMMC/holdings-in-company-5g3485s1dpu5hct.html
Found 'Saba' in: https://www.lse.co.uk/rns/SCP/holdings-in-company-g205l7fud9t1idp.html
Found 'Saba' in: https://www.lse.co.uk/rns/USA/total-voting-rights-iep6fwf6w56vpy5.html


In [4]:
results

{'JMG': False,
 'JUGI': True,
 'JUSC': False,
 'KPC': True,
 'LTI': False,
 'LWDB': False,
 'LWI': True,
 'MCT': True,
 'MHN': False,
 'MIGO': False,
 'MINI': False,
 'MIX': False,
 'MMIT': False,
 'MNKS': False,
 'MNL': False,
 'MNP': False,
 'MRC': False,
 'MRCH': False,
 'MTE': True,
 'MTU': True,
 'MUT': False,
 'MWY': False,
 'MYI': False,
 'NAIT': True,
 'NAS': False,
 'NAVF': False,
 'NSI': False,
 'OIG': False,
 'OIT': False,
 'ONWD': False,
 'OOA': False,
 'OSEC': False,
 'OVCT': False,
 'PAC': False,
 'PCFT': False,
 'PCGH': False,
 'PCT': False,
 'PGV': False,
 'PHI': False,
 'PNL': False,
 'PSH': False,
 'RCOI': False,
 'RICA': False,
 'RIII': False,
 'RKW': False,
 'RMMC': True,
 'SAIN': False,
 'SCF': False,
 'SCP': True,
 'SDP': False,
 'SDV': False,
 'SEC': False,
 'SHRS': False,
 'SIHL': False,
 'SJG': False,
 'SMT': False,
 'SOI': False,
 'SSON': False,
 'SST': False,
 'STS': False,
 'SVM': False,
 'TEM': False,
 'TENT': False,
 'THRG': False,
 'TMPL': False,
 'TRY': 